# notebook/03_data_transformation_to_gold

In [1]:
import os
import sys 
import logging

# Caminho para a raiz do seu projeto
project_root = os.path.abspath("..")  # Sobe um nível de pasta a partir de 'notebooks'

# Adiciona o caminho ao sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.utils import functions as F
from config.configs import *
from config.spark_session import create_spark_session

## Spark Session

In [ ]:
spark = create_spark_session(HOST_ADDRESS, MINIO_ACCESS_KEY, MINIO_SECRET_KEY)

## Log configs

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

logging.info("Starting convertions from Silver(Delta) to Gold(Delta) - COVID19...")

2024-11-18 11:42:55,419 - INFO - Starting convertions from Silver(Delta) to Gold(Delta) - COVID19...


## Path configs

In [4]:
input_prefix_layer_name = prefix_layer_name['2']  # silver layer
input_path = lake_path['silver']

output_prefix_layer_name = prefix_layer_name['3']  # gold layer
storage_output = lake_path['gold']
output_tabela_name = tabela_name['0']  # nivel agregação

output_full_name = f"{storage_output}{output_prefix_layer_name}{output_tabela_name}"

## Process

In [ ]:
try:
    for table_name, query_input in tables_gold.items():
        table_name = F.convert_table_name(table_name)

        # Obter a consulta para o carregamento do arquivo
        query_input = F.get_query(table_name, input_path, input_prefix_layer_name, output_tabela_name, tables_gold)
        
        # Definir o caminho de saída
        storage_output = f'{output_full_name}'

        # Chamar a função process_table para processar e gravar os dados
        F.process_table(spark, query_input, output_full_name)
    
    logging.info("Process to silver completed!")
    
except Exception as e:
    logging.error(f'Error processing table: {str(e)}')